https://zhuanlan.zhihu.com/p/26951125

![](https://pic2.zhimg.com/v2-a325a6cd9f97771071db9a864f535669_1200x500.jpg)

In [19]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

In [20]:
# 定义超参数
batch_size = 128
learning_rate = 1e-2
num_epoches = 20

In [21]:
def to_np(x):
    return x.cpu().data.numpy()

In [22]:
# 下载训练集 MNIST 手写数字训练集
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [23]:
# 定义 Convolution Network 模型
class Cnn(nn.Module):
    def __init__(self, in_dim, n_class):
        super(Cnn, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_dim, 6, 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5, stride=1, padding=0),
            nn.ReLU(True), nn.MaxPool2d(2, 2))

        self.fc = nn.Sequential(
            nn.Linear(400, 120), nn.Linear(120, 84), nn.Linear(84, n_class))

    def forward(self, x):
        out = self.conv(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [24]:
model = Cnn(1, 10)  # 图片大小是28x28
use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速
if use_gpu:
    model = model.cuda()
# 定义loss和optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [25]:
for epoch in range(num_epoches):
    print('epoch {}'.format(epoch + 1))
    print('*' * 10)
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        if use_gpu:
            img = img.cuda()
            label = label.cuda()
        img = Variable(img)
        label = Variable(label)
        # 向前传播
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss.item() * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        accuracy = (pred == label).float().mean()
        running_acc += num_correct.item()
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

        if i % 300 == 0:
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(epoch + 1, num_epoches, running_loss / (batch_size * i),
                running_acc / (batch_size * i)))
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, running_loss / (len(train_dataset)), running_acc / (len(train_dataset))))
    model.eval()
    eval_loss = 0
    eval_acc = 0
    for data in test_loader:
        img, label = data
        if use_gpu:
            with torch.no_grad():
                img = img.cuda()            
                label = label.cuda()
        else:
            with torch.no_grad():
                img = img            
                label = label
        out = model(img)
        loss = criterion(out, label)
        eval_loss += loss.item() * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        eval_acc += num_correct.item()
    print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(test_dataset)), eval_acc / (len(test_dataset))))
    print()

epoch 1
**********
[1/20] Loss: 2.296549, Acc: 0.104740
Finish 1 epoch, Loss: 2.280768, Acc: 0.204467
Test Loss: 2.201868, Acc: 0.482500

epoch 2
**********
[2/20] Loss: 1.506672, Acc: 0.643672
Finish 2 epoch, Loss: 1.149842, Acc: 0.718283
Test Loss: 0.418522, Acc: 0.877200

epoch 3
**********
[3/20] Loss: 0.376417, Acc: 0.888698
Finish 3 epoch, Loss: 0.354870, Acc: 0.895000
Test Loss: 0.279139, Acc: 0.917000

epoch 4
**********
[4/20] Loss: 0.290326, Acc: 0.913620
Finish 4 epoch, Loss: 0.277530, Acc: 0.917017
Test Loss: 0.228941, Acc: 0.931300

epoch 5
**********
[5/20] Loss: 0.239458, Acc: 0.928568
Finish 5 epoch, Loss: 0.229221, Acc: 0.931417
Test Loss: 0.186778, Acc: 0.944200

epoch 6
**********
[6/20] Loss: 0.196174, Acc: 0.940286
Finish 6 epoch, Loss: 0.189857, Acc: 0.942683
Test Loss: 0.157567, Acc: 0.953600

epoch 7
**********
[7/20] Loss: 0.168095, Acc: 0.949948
Finish 7 epoch, Loss: 0.161158, Acc: 0.952167
Test Loss: 0.141814, Acc: 0.954300

epoch 8
**********
[8/20] Loss: 0.